In [1]:
import pandas as pd
import datetime

In [2]:
jailed_date = datetime.datetime.strptime('2023-03-22', "%Y-%m-%d")

## Prepairing norpadon csv

In [3]:
norpadon_df = pd.read_csv('norpadon corpus/fbk_archive.csv', names=['author', 'text', 'date', 'url', 'source_type'])

In [4]:
norpadon_df['date'] = norpadon_df['date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))

In [5]:
norpadon_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39251 entries, 0 to 39250
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   author       39251 non-null  object        
 1   text         39242 non-null  object        
 2   date         39251 non-null  datetime64[ns]
 3   url          39251 non-null  object        
 4   source_type  39251 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 1.5+ MB


In [6]:
dates = ['free' if date < jailed_date else 'jailed'
         for date in norpadon_df['date']]

In [7]:
norpadon_df['befor/afrer jail'] = dates

In [8]:
norpadon_df.head()

,author,text,date,url,source_type,befor/afrer jail
0,Navalny,I wanted to write about this for a long time. ...,2023-08-11,https://navalny.com/p/6652/,https://navalny.com,jailed
1,Navalny,"Давно собирался написать об этом. Ну, пусть бу...",2023-08-11,https://navalny.com/p/6651/,https://navalny.com,jailed
2,Navalny,"Знакомьтесь, это Терентьев Сергей Андреевич.Во...",2023-08-09,https://navalny.com/p/6650/,https://navalny.com,jailed
3,Navalny,Сегодня Алексею Навальному вынесут приговор. П...,2023-08-04,https://navalny.com/p/6649/,https://navalny.com,jailed
4,Navalny,The sentence will be read in twenty-four hours...,2023-08-03,https://navalny.com/p/6648/,https://navalny.com,jailed


In [9]:
norpadon_df.author.unique()

array(['Navalny', 'Volkov', 'Sobol', 'Pevchikh', 'Alburov', 'Zhdanov',
       'Yarmysh'], dtype=object)

In [10]:
corpus = norpadon_df.drop(['date', 'url'], axis=1)

In [11]:
corpus['team'] = ['norpadon' for _ in range(len(corpus))]

In [12]:
webblogs = [source for source in corpus.source_type.unique() 
            if source.split('/')[0] not in ['t.me', 'instagram.com', 'facebook.com']]
webblogs

['https://navalny.com',
 'https://leonwolf.livejournal.com',
 'https://sobollubov.livejournal.com']

In [13]:
corpus = corpus[(corpus['source_type'] != 'https://navalny.com') & 
                (corpus['source_type'] != 'https://leonwolf.livejournal.com') & 
                (corpus['source_type'] != 'https://sobollubov.livejournal.com')]

In [14]:
corpus.source_type.unique()

array(['t.me/navalny', 'instagram.com/navalny', 't.me/leonid_volkov',
       'instagram.com/leonidvolkov', 't.me/TeamSobol',
       'instagram.com/sobollubov', 'instagram.com/maria_pevchikh',
       'instagram.com/alburov', 't.me/ioannzh', 'instagram.com/ioannzh',
       't.me/Kira_Yarmysh', 'instagram.com/kira_yarmysh',
       'facebook.com/navalny', 'facebook.com/leonid.m.volkov',
       'facebook.com/soboll.ru', 'facebook.com/alburov',
       'facebook.com/zhdanovivan'], dtype=object)

In [15]:
corpus

,author,text,source_type,befor/afrer jail,team
2978,Navalny,"Вчера ракетостроитель Борис Обносов, его дочь ...",t.me/navalny,jailed,norpadon
2979,Navalny,"Давно собирался написать об этом. Ну, пусть бу...",t.me/navalny,jailed,norpadon
2980,Navalny,❤️‍🩹Письмо от Даниэля Холодного:Всем привет! Э...,t.me/navalny,jailed,norpadon
2981,Navalny,19 лет колонии особого режима. Цифра не имеет ...,t.me/navalny,jailed,norpadon
2982,Navalny,"Все, что происходит с Алексеем Навальным, — эт...",t.me/navalny,jailed,norpadon
...,...,...,...,...,...
39246,Zhdanov,"Настанет день, и с журавлиной стаей Я поплыву ...",facebook.com/zhdanovivan,free,norpadon
39247,Zhdanov,Всё-таки опыт! честно говоря перечитывал приго...,facebook.com/zhdanovivan,free,norpadon
39248,Zhdanov,"Песков классный.\nИлья Пономарев:\n""Спросил у ...",facebook.com/zhdanovivan,free,norpadon
39249,Zhdanov,Even squirrels sleep sound in the arms of Ange...,facebook.com/zhdanovivan,free,norpadon


## Prepairing sysblock texts

In [16]:
dataframes = [corpus]
columns = ['author', 'text', 'source_type', 'befor/afrer jail', 'team']

In [17]:
with open('sysblock corpus/alburov_twitter.txt', encoding='utf-8') as f:
    texts = [line.strip() for line in f.read().split('\n\n') if line != '']

In [18]:
alburov_twitter = pd.DataFrame(zip(['Alburov' for _ in range(len(texts))], 
                                texts, 
                                ['twitter.com/alburov' for _ in range(len(texts))],  
                                ['jailed' for _ in range(len(texts))], 
                                ['sysblock' for _ in range(len(texts))]), columns=columns)

dataframes.append(alburov_twitter)

In [19]:
alburov_twitter

,author,text,source_type,befor/afrer jail,team
0,Alburov,"﻿Это самое большое расследование, которое мы к...",twitter.com/alburov,jailed,sysblock
1,Alburov,Нашего дорого Ивана @ioannZH с днем рождения!,twitter.com/alburov,jailed,sysblock
2,Alburov,"Мудрость Ивана доказывается хотя бы тем, что о...",twitter.com/alburov,jailed,sysblock
3,Alburov,"О, с основного аккаунта, спасибо!",twitter.com/alburov,jailed,sysblock
4,Alburov,"Блин, не тот аккаунт",twitter.com/alburov,jailed,sysblock
...,...,...,...,...,...
219,Alburov,"Бойкот авторов, которые призывают переходить и...",twitter.com/alburov,jailed,sysblock
220,Alburov,Знакомые сделали фонд и помогают беженцам из У...,twitter.com/alburov,jailed,sysblock
221,Alburov,Алексей Навальный незаконно находится в тюрьме...,twitter.com/alburov,jailed,sysblock
222,Alburov,Как же мощно Маск борется с шэдоу банами. Тепе...,twitter.com/alburov,jailed,sysblock


In [20]:
with open('sysblock corpus/free_navalny.txt', encoding='utf-8') as f:
    texts = [line.strip() for line in f.read().split('\n') 
             if line != '⠀' and line not in corpus[corpus['author'] == 'Navalny']['text'].tolist()]

In [21]:
free_navalny = pd.DataFrame(zip(['Navalny' for _ in range(len(texts))], 
                                  texts, 
                                  ['unknown' for _ in range(len(texts))],  
                                  ['free' for _ in range(len(texts))], 
                                  ['sysblock' for _ in range(len(texts))]), columns=columns)

dataframes.append(free_navalny)

In [22]:
free_navalny

,author,text,source_type,befor/afrer jail,team
0,Navalny,Все мы находимся в плену стереотипов. В том чи...,unknown,free,sysblock
1,Navalny,"Все пять месяцев, которые я здесь пробыл, я по...",unknown,free,sysblock
2,Navalny,Вообще не так. Это действительно милейшие люди...,unknown,free,sysblock
3,Navalny,- немцы не опаздывают (это приводило меня в во...,unknown,free,sysblock
4,Navalny,- немцы действительно обожают придумывать прав...,unknown,free,sysblock
...,...,...,...,...,...
301,Navalny,"Мы туда поехали пораньше, пока мало народу.",unknown,free,sysblock
302,Navalny,"Я бегал, а Юля с Захаром, приунывшие от карант...",unknown,free,sysblock
303,Navalny,"Маски надевали, если рядом какие-то люди шли.",unknown,free,sysblock
304,Navalny,"Ну так вот: народ реально шарахается от тех, к...",unknown,free,sysblock


In [23]:
with open('sysblock corpus/free_navalny_2.txt', encoding='utf-8') as f:
    texts = [line.strip() for line in f.read().split('\n') 
             if line != '' and line not in corpus[corpus['author'] == 'Navalny']['text'].tolist()]

In [24]:
free_navalny_2 = pd.DataFrame(zip(['Navalny' for _ in range(len(texts))], 
                                  texts, 
                                  ['unknown' for _ in range(len(texts))],  
                                  ['free' for _ in range(len(texts))], 
                                  ['sysblock' for _ in range(len(texts))]), columns=columns)

dataframes.append(free_navalny_2)

In [25]:
free_navalny_2

,author,text,source_type,befor/afrer jail,team
0,Navalny,«Владимир Путин сообщил французскому коллеге: ...,unknown,free,sysblock
1,Navalny,"Хорошая версия. Считаю, что заслуживает самого...",unknown,free,sysblock
2,Navalny,Сварил на кухне «Новичок». Тихо отхлебнул из ф...,unknown,free,sysblock
3,Navalny,"Его просто так не проведёшь. В итоге я, как ду...",unknown,free,sysblock
4,Navalny,Пост про любовь.,unknown,free,sysblock
...,...,...,...,...,...
327,Navalny,юбая наша поездка уже давно проходит в самом р...,unknown,free,sysblock
328,Navalny,Ну и так далее. Часть их важной «государственн...,unknown,free,sysblock
329,Navalny,А там уже делают репортаж о моей роскошной жиз...,unknown,free,sysblock
330,Navalny,"Перелёт становится «бизнес-классом», стоимость...",unknown,free,sysblock


In [26]:
with open('sysblock corpus/jailed_navalny.txt', encoding='utf-8') as f:
    texts = [line.strip() for line in f.read().split('\n') 
             if line != '' and line not in corpus[corpus['author'] == 'Navalny']['text'].tolist()]

In [27]:
jailed_navalny = pd.DataFrame(zip(['Navalny' for _ in range(len(texts))], 
                                  texts, 
                                  ['unknown' for _ in range(len(texts))], 
                                  ['jailed' for _ in range(len(texts))], 
                                  ['sysblock' for _ in range(len(texts))]),  columns=columns)


dataframes.append(jailed_navalny)

In [28]:
jailed_navalny

,author,text,source_type,befor/afrer jail,team
0,Navalny,"Давно собирался написать об этом. Ну, пусть бу...",unknown,jailed,sysblock
1,Navalny,"Ненависть. Об этом всегда спрашивают много, и ...",unknown,jailed,sysblock
2,Navalny,"Но, признаюсь честно, есть она у меня. И ещё к...",unknown,jailed,sysblock
3,Navalny,Бешусь я страшно после заседаний местного райс...,unknown,jailed,sysblock
4,Navalny,"Но хоть я иногда, не выдерживая, ору на какого...",unknown,jailed,sysblock
...,...,...,...,...,...
180,Navalny,"Под шквальным огнем им пришлось выживать,",unknown,jailed,sysblock
181,Navalny,"Какая-то крыса орудует в штабе,",unknown,jailed,sysblock
182,Navalny,"И жаль, что пока ее не достать».",unknown,jailed,sysblock
183,Navalny,"Я говорю ментам: вы вообще слушаете, что у вас...",unknown,jailed,sysblock


In [29]:
with open('sysblock corpus/jailed_navalny_2.txt', encoding='utf-8') as f:
    texts = [line.strip() for line in f.read().split('\n') 
             if line != '' and line not in corpus[corpus['author'] == 'Navalny']['text'].tolist()]

In [30]:
jailed_navalny_2 = pd.DataFrame(zip(['Navalny' for _ in range(len(texts))], 
                                  texts, 
                                  ['unknown' for _ in range(len(texts))],   
                                  ['jailed' for _ in range(len(texts))], 
                                  ['sysblock' for _ in range(len(texts))]), columns=columns)

In [31]:
jailed_navalny_2

,author,text,source_type,befor/afrer jail,team
0,Navalny,"Догадайтесь, кто чемпион по прослушиванию рече...",unknown,jailed,sysblock
1,Navalny,"Я, конечно.",unknown,jailed,sysblock
2,Navalny,Давно в каком-то шпионском детективе я читал о...,unknown,jailed,sysblock
3,Navalny,"Видимо, кто-то в нашей тюремной системе тоже ч...",unknown,jailed,sysblock
4,Navalny,"После того как мы опубликовали историю о том, ...",unknown,jailed,sysblock
...,...,...,...,...,...
171,Navalny,"Вынужден признаться, что идея работает: ни ску...",unknown,jailed,sysblock
172,Navalny,"Но, как обычно, в таких ситуациях меня изумляе...",unknown,jailed,sysblock
173,Navalny,"У них же план был, понимаете. Просто так зека ...",unknown,jailed,sysblock
174,Navalny,А мораль истории простая: тюремной системой Ро...,unknown,jailed,sysblock


In [32]:
with open('sysblock corpus/navalnaya_inst.txt', encoding='utf-8') as f:
    texts = [line.strip() for line in f.read().split('\n') 
             if line != '']

In [33]:
navalnaya_inst = pd.DataFrame(zip(['Navalnaya' for _ in range(len(texts))], 
                                  texts, 
                                  ['instagram.com/yulia_navalnaya' for _ in range(len(texts))],  
                                  ['unknown' for _ in range(len(texts))], 
                                  ['sysblock' for _ in range(len(texts))]), columns=columns)

dataframes.append(navalnaya_inst)

In [34]:
navalnaya_inst

,author,text,source_type,befor/afrer jail,team
0,Navalnaya,В 5 часов утра получила на смятом клочке бумаг...,instagram.com/yulia_navalnaya,unknown,sysblock
1,Navalnaya,"У всех бывают трудные времена. Наверное, у нас...",instagram.com/yulia_navalnaya,unknown,sysblock
2,Navalnaya,"Но я знаю, и мне не надо у себя убеждать, что ...",instagram.com/yulia_navalnaya,unknown,sysblock
3,Navalnaya,Только чтобы на таких фотографиях нас было сно...,instagram.com/yulia_navalnaya,unknown,sysblock
4,Navalnaya,Спасибо вам огромное за поздравления. Я это оч...,instagram.com/yulia_navalnaya,unknown,sysblock
...,...,...,...,...,...
199,Navalnaya,"Какое счастье, что вы все здесь! Спасибо!",instagram.com/yulia_navalnaya,unknown,sysblock
200,Navalnaya,Мне в жизни очень повезло дважды. Это как выиг...,instagram.com/yulia_navalnaya,unknown,sysblock
201,Navalnaya,"Ровняясь на него все эти годы, я не могу вести...",instagram.com/yulia_navalnaya,unknown,sysblock
202,Navalnaya,"Уже 22 года абсолютного счастья. Леша, ты моя ...",instagram.com/yulia_navalnaya,unknown,sysblock


In [35]:
with open('sysblock corpus/pevchikh_inst_1.txt', encoding='utf-8') as f:
    texts = [line.strip() for line in f.read().split('\n\n') 
             if line != '']

In [36]:
pevchikh_inst_1 = pd.DataFrame(zip(['Pevchikh' for _ in range(len(texts))], 
                                  texts, 
                                  ['instagram.com/maria_pevchikh' for _ in range(len(texts))],  
                                  ['unknown' for _ in range(len(texts))], 
                                  ['sysblock' for _ in range(len(texts))]), columns=columns)

dataframes.append(pevchikh_inst_1)

In [37]:
pevchikh_inst_1

,author,text,source_type,befor/afrer jail,team
0,Pevchikh,"""День 16. Счастливая дама с табличкой.",instagram.com/maria_pevchikh,unknown,sysblock
1,Pevchikh,"Мало тех, кого может порадовать табличка в экс...",instagram.com/maria_pevchikh,unknown,sysblock
2,Pevchikh,"Когда мы делаем и публикуем расследование, нас...",instagram.com/maria_pevchikh,unknown,sysblock
3,Pevchikh,"В случае Обносова и Зориковых, работать пришло...",instagram.com/maria_pevchikh,unknown,sysblock
4,Pevchikh,Ну и самый главный ингредиент. Вы. И ваша помо...,instagram.com/maria_pevchikh,unknown,sysblock
...,...,...,...,...,...
191,Pevchikh,"😘""\n""Вела в понедельник новостной эфир и получ...",instagram.com/maria_pevchikh,unknown,sysblock
192,Pevchikh,"Ну вот, смотрите, что происходит на самом деле...",instagram.com/maria_pevchikh,unknown,sysblock
193,Pevchikh,"И, как лучшие инстаграм коучи из рилзов, я про...",instagram.com/maria_pevchikh,unknown,sysblock
194,Pevchikh,А потом начинается эфир. Сначала как-то тоскли...,instagram.com/maria_pevchikh,unknown,sysblock


In [38]:
with open('sysblock corpus/pevchikh_inst_2.txt', encoding='utf-8') as f:
    texts = [line.strip() for line in f.read().split('\n\n') 
             if line != '']

In [39]:
pevchikh_inst_2 = pd.DataFrame(zip(['Pevchikh' for _ in range(len(texts))], 
                                  texts, 
                                  ['instagram.com/maria_pevchikh' for _ in range(len(texts))], 
                                  ['unknown' for _ in range(len(texts))], 
                                  ['sysblock' for _ in range(len(texts))]), columns=columns)

dataframes.append(pevchikh_inst_2)

In [40]:
pevchikh_inst_2

,author,text,source_type,befor/afrer jail,team
0,Pevchikh,"""Сегодня лучший день, чтобы поздравить с днем ...",instagram.com/maria_pevchikh,unknown,sysblock
1,Pevchikh,"Посмотрите на фото, этому веселому парню с так...",instagram.com/maria_pevchikh,unknown,sysblock
2,Pevchikh,"Спасибо тебе, Жорес, ты уже так много сделал, ...",instagram.com/maria_pevchikh,unknown,sysblock
3,Pevchikh,"С днем рождения!""\n""Как же меня бесит такое, п...",instagram.com/maria_pevchikh,unknown,sysblock
4,Pevchikh,"А чиновница, которая отвечает за все это, скуп...",instagram.com/maria_pevchikh,unknown,sysblock
...,...,...,...,...,...
191,Pevchikh,"Специально прикрепляю видео, чтобы вы оценили,...",instagram.com/maria_pevchikh,unknown,sysblock
192,Pevchikh,А слева - головорез Иван Жданов (в преступных ...,instagram.com/maria_pevchikh,unknown,sysblock
193,Pevchikh,Очень опасные люди. Лучшие коллеги на свете. ❤️,instagram.com/maria_pevchikh,unknown,sysblock
194,Pevchikh,"P. S. В качестве бонуса, чтобы вы подписывалис...",instagram.com/maria_pevchikh,unknown,sysblock


In [41]:
with open('sysblock corpus/Shaveddinov_insta.txt', encoding='utf-8') as f:
    texts = [line.strip() for line in f.read().split('\n\n') 
             if line != ' ' and line not in corpus[corpus['author'] == 'Shaveddinov']['text'].tolist()]

In [42]:
Shaveddinov_insta = pd.DataFrame(zip(['Shaveddinov' for _ in range(len(texts))], 
                                  texts, 
                                  ['instagram.com/rshaveddinov' for _ in range(len(texts))], 
                                  ['unknown' for _ in range(len(texts))], 
                                  ['sysblock' for _ in range(len(texts))]), columns=columns)

dataframes.append(Shaveddinov_insta)

In [43]:
Shaveddinov_insta

,author,text,source_type,befor/afrer jail,team
0,Shaveddinov,"Как известно, чтобы стать бегуном нужно:\n1) к...",instagram.com/rshaveddinov,unknown,sysblock
1,Shaveddinov,Наличие этих атрибутов сразу переводит вас в л...,instagram.com/rshaveddinov,unknown,sysblock
2,Shaveddinov,Бегал сегодня первый ParkRun после долгого пер...,instagram.com/rshaveddinov,unknown,sysblock
3,Shaveddinov,"Все еще тяжко, но почему-то хочется верить, чт...",instagram.com/rshaveddinov,unknown,sysblock
4,Shaveddinov,"Бегать я пытаюсь, чтобы хоть какая-то физическ...",instagram.com/rshaveddinov,unknown,sysblock
...,...,...,...,...,...
75,Shaveddinov,"с днём рождения, Алексей @navalny!\nЗдоровья и...",instagram.com/rshaveddinov,unknown,sysblock
76,Shaveddinov,Записали с @leonidvolkov и @maria_pevchikh нов...,instagram.com/rshaveddinov,unknown,sysblock
77,Shaveddinov,Улица небезопасная штука. Идёшь себе такой вес...,instagram.com/rshaveddinov,unknown,sysblock
78,Shaveddinov,АРЕСТОВАЛИ ЯХТУ ПУТИНА\nУ нас за спиной $700 м...,instagram.com/rshaveddinov,unknown,sysblock


In [44]:
with open('sysblock corpus/Shaveddinov_twitter.txt', encoding='utf-8') as f:
    texts = [line.strip() for line in f.read().split('\n\n') 
             if line != ' ']

In [45]:
Shaveddinov_twitter = pd.DataFrame(zip(['Shaveddinov' for _ in range(len(texts))], 
                                  texts, 
                                  ['twitter.com/rshaveddinov' for _ in range(len(texts))],  
                                  ['jailed' for _ in range(len(texts))], 
                                  ['sysblock' for _ in range(len(texts))]), columns=columns)

dataframes.append(Shaveddinov_twitter)

In [46]:
Shaveddinov_twitter

,author,text,source_type,befor/afrer jail,team
0,Shaveddinov,Самая важная ссылка. Помогите максимально расп...,twitter.com/rshaveddinov,jailed,sysblock
1,Shaveddinov,Путинская пропаганда над которой мы тут с вами...,twitter.com/rshaveddinov,jailed,sysblock
2,Shaveddinov,После мощного общественного давления власти Че...,twitter.com/rshaveddinov,jailed,sysblock
3,Shaveddinov,В эфире на Популярной политике сейчас Роман До...,twitter.com/rshaveddinov,jailed,sysblock
4,Shaveddinov,"Важно помнить: пока Путин жив, никто не в безо...",twitter.com/rshaveddinov,jailed,sysblock
...,...,...,...,...,...
199,Shaveddinov,Важные истории публикует огромное расследовани...,twitter.com/rshaveddinov,jailed,sysblock
200,Shaveddinov,,twitter.com/rshaveddinov,jailed,sysblock
201,Shaveddinov,,twitter.com/rshaveddinov,jailed,sysblock
202,Shaveddinov,,twitter.com/rshaveddinov,jailed,sysblock


In [47]:
with open('sysblock corpus/Sobol_twi_Aug23.txt', encoding='utf-8') as f:
    texts = [line.strip() for line in f.read().split('\n\n') 
             if line != ' ']

In [48]:
Sobol_twi_Aug23 = pd.DataFrame(zip(['Sobol' for _ in range(len(texts))], 
                                  texts, 
                                  ['twitter.com/SobolLubov' for _ in range(len(texts))], 
                                  ['jailed' for _ in range(len(texts))], 
                                  ['sysblock' for _ in range(len(texts))]), columns=columns)

dataframes.append(Sobol_twi_Aug23)

In [49]:
Sobol_twi_Aug23

,author,text,source_type,befor/afrer jail,team
0,Sobol,На Радио Свобода обсуждали пост Навального -ые...,twitter.com/SobolLubov,jailed,sysblock
1,Sobol,Считаю конечно что путинизм появился не на пус...,twitter.com/SobolLubov,jailed,sysblock
2,Sobol,Еще как по мне в -ые не провели главную реформ...,twitter.com/SobolLubov,jailed,sysblock
3,Sobol,В общем спор содержательный вышел послушайте \...,twitter.com/SobolLubov,jailed,sysblock
4,Sobol,Когда я решу рассказать об этом и опубличить д...,twitter.com/SobolLubov,jailed,sysblock
...,...,...,...,...,...
212,Sobol,С днем рождения друг,twitter.com/SobolLubov,jailed,sysblock
213,Sobol,Прощение любовь обогащение всех а не избранных...,twitter.com/SobolLubov,jailed,sysblock
214,Sobol,Постпутинская Россия должна быть колыбелью под...,twitter.com/SobolLubov,jailed,sysblock
215,Sobol,Еще важно отсутствие возмездия Это то что Нава...,twitter.com/SobolLubov,jailed,sysblock


In [50]:
with open('sysblock corpus/volkov_twitter.txt', encoding='utf-8') as f:
    texts = [line.strip() for line in f.read().split('\n\n') 
             if line != ' ']

In [51]:
volkov_twitter = pd.DataFrame(zip(['Volkov' for _ in range(len(texts))], 
                                  texts, 
                                  ['twitter.com/leonidvolkov' for _ in range(len(texts))],   
                                  ['jailed' for _ in range(len(texts))], 
                                  ['sysblock' for _ in range(len(texts))]), columns=columns)

dataframes.append(volkov_twitter)

In [52]:
volkov_twitter

,author,text,source_type,befor/afrer jail,team
0,Volkov,"Грише Мелконьянцу и всем его коллегам из ""Голо...",twitter.com/leonidvolkov,jailed,sysblock
1,Volkov,"Приходите на эфир, буду отвечать на вопросы! \...",twitter.com/leonidvolkov,jailed,sysblock
2,Volkov,"Ракетостроитель Борис Обносов, его дочь и ее м...",twitter.com/leonidvolkov,jailed,sysblock
3,Volkov,Это не символический шаг. Зориковы живут в Пра...,twitter.com/leonidvolkov,jailed,sysblock
4,Volkov,"Отдельный привет всем, кто говорил про ""беспол...",twitter.com/leonidvolkov,jailed,sysblock
...,...,...,...,...,...
94,Volkov,Это звучит как начало анекдота. \nТипа заходят...,twitter.com/leonidvolkov,jailed,sysblock
95,Volkov,Наша итоговая недельная передача — повод для г...,twitter.com/leonidvolkov,jailed,sysblock
96,Volkov,Фан факт: генералы армии Суровикин и Золотов п...,twitter.com/leonidvolkov,jailed,sysblock
97,Volkov,Я сегодня в вечернем информационном эфире — ме...,twitter.com/leonidvolkov,jailed,sysblock


In [53]:
with open('sysblock corpus/zhdanov_twitter.txt', encoding='utf-8') as f:
    texts = [line.strip() for line in f.read().split('\n\n') 
             if line != ' ']

In [54]:
zhdanov_twitter = pd.DataFrame(zip(['Zhdanov' for _ in range(len(texts))], 
                                  texts, 
                                  ['twitter.com/IoannZH' for _ in range(len(texts))], 
                                  ['jailed' for _ in range(len(texts))], 
                                  ['sysblock' for _ in range(len(texts))]), columns=columns)

dataframes.append(zhdanov_twitter)

In [55]:
zhdanov_twitter

,author,text,source_type,befor/afrer jail,team
0,Zhdanov,Задача про синюю и красную кнопки — это же сов...,twitter.com/IoannZH,jailed,sysblock
1,Zhdanov,"Очевидно, что ФБК ничего не решает и никакого ...",twitter.com/IoannZH,jailed,sysblock
2,Zhdanov,После запуска чатов на платформе Штабов Наваль...,twitter.com/IoannZH,jailed,sysblock
3,Zhdanov,Всем привет. Это Жора Албуров. Сегодня меня по...,twitter.com/IoannZH,jailed,sysblock
4,Zhdanov,Как же Ксения Анатольевна расстроилась из-за к...,twitter.com/IoannZH,jailed,sysblock
...,...,...,...,...,...
152,Zhdanov,"Напоминаю. 15 пунктов гражданина России, желаю...",twitter.com/IoannZH,jailed,sysblock
153,Zhdanov,Вообще я против удаления и блокировок любых ка...,twitter.com/IoannZH,jailed,sysblock
154,Zhdanov,Чуть не пропустил. Ровно год с первого стрима ...,twitter.com/IoannZH,jailed,sysblock
155,Zhdanov,А потом еще несколько сумасшедших многочасовых...,twitter.com/IoannZH,jailed,sysblock


In [56]:
with open('sysblock corpus/yarmysh_1.txt', encoding='utf-8') as f:
    texts = [line.strip() for line in f.read().split('\n\n') 
             if line != ' ' and line not in corpus[corpus['author'] == 'Yarmysh']['text'].tolist()]

In [57]:
yarmysh_1 = pd.DataFrame(zip(['Yarmysh' for _ in range(len(texts))], 
                                  texts, 
                                  ['unknown' for _ in range(len(texts))],  
                                  ['unknown' for _ in range(len(texts))], 
                                  ['sysblock' for _ in range(len(texts))]), columns=columns)

dataframes.append(yarmysh_1)

In [58]:
yarmysh_1

,author,text,source_type,befor/afrer jail,team
0,Yarmysh,[a]\nЯ не заготовила на сегодня никакого пафос...,unknown,unknown,sysblock
1,Yarmysh,Приговор зачитали на час позже назначенного ср...,unknown,unknown,sysblock
2,Yarmysh,У меня нет иллюзий насчет судов в России. Я са...,unknown,unknown,sysblock
3,Yarmysh,Это же не только с судами так. Все государстве...,unknown,unknown,sysblock
4,Yarmysh,"И несмотря на то, что это приводит меня в беше...",unknown,unknown,sysblock
...,...,...,...,...,...
225,Yarmysh,Это не солдаты. Они не погибли случайно под от...,unknown,unknown,sysblock
226,Yarmysh,Когда Симоньян и Соловьев радовались - «начало...,unknown,unknown,sysblock
227,Yarmysh,"Путин, отдавший приказ, - преступник. Я понима...",unknown,unknown,sysblock
228,Yarmysh,"Военные, которые выполнили путинский приказ, -...",unknown,unknown,sysblock


## Corpus

In [59]:
corpus = pd.concat(dataframes).dropna()

In [60]:
corpus = corpus.reset_index(drop=True)

In [61]:
corpus

,author,text,source_type,befor/afrer jail,team
0,Navalny,"Вчера ракетостроитель Борис Обносов, его дочь ...",t.me/navalny,jailed,norpadon
1,Navalny,"Давно собирался написать об этом. Ну, пусть бу...",t.me/navalny,jailed,norpadon
2,Navalny,❤️‍🩹Письмо от Даниэля Холодного:Всем привет! Э...,t.me/navalny,jailed,norpadon
3,Navalny,19 лет колонии особого режима. Цифра не имеет ...,t.me/navalny,jailed,norpadon
4,Navalny,"Все, что происходит с Алексеем Навальным, — эт...",t.me/navalny,jailed,norpadon
...,...,...,...,...,...
38266,Yarmysh,Это не солдаты. Они не погибли случайно под от...,unknown,unknown,sysblock
38267,Yarmysh,Когда Симоньян и Соловьев радовались - «начало...,unknown,unknown,sysblock
38268,Yarmysh,"Путин, отдавший приказ, - преступник. Я понима...",unknown,unknown,sysblock
38269,Yarmysh,"Военные, которые выполнили путинский приказ, -...",unknown,unknown,sysblock


In [62]:
corpus.author.unique()

array(['Navalny', 'Volkov', 'Sobol', 'Pevchikh', 'Alburov', 'Zhdanov',
       'Yarmysh', 'Navalnaya', 'Shaveddinov'], dtype=object)

In [64]:
corpus[corpus['author'] == 'Sobol'].shape

(17688, 5)

In [74]:
corpus.to_csv('fbk_corpus.csv', index=False)